In [ ]:
import pandas as pd
import numpy as np
import joblib
import re

# ============================================================
# 🔧 Função auxiliar
# ============================================================
def extrair_numero(texto):
    if pd.isna(texto):
        return np.nan
    numeros = re.findall(r'\d+', str(texto))
    return int(numeros[0]) if numeros else np.nan


# ============================================================
# 📊 Estatísticas descritivas
# ============================================================
def estatisticas_descritivas(df):
    col_num = df.select_dtypes(include=['number']).columns
    col_cat = df.select_dtypes(include=['object', 'category']).columns
    col_date = df.select_dtypes(include=['datetime64[ns]', 'datetime64']).columns

    resumo = {}

    if len(col_num) > 0:
        stats_num = pd.DataFrame(index=col_num)
        stats_num['Count'] = df[col_num].count()
        stats_num['Mean'] = df[col_num].mean()
        stats_num['Std'] = df[col_num].std()
        stats_num['Var'] = df[col_num].var()
        stats_num['Min'] = df[col_num].min()
        stats_num['25%'] = df[col_num].quantile(0.25)
        stats_num['50% (Median)'] = df[col_num].median()
        stats_num['75%'] = df[col_num].quantile(0.75)
        stats_num['Max'] = df[col_num].max()
        resumo['Numéricas'] = stats_num.round(3)

    if len(col_cat) > 0:
        stats_cat = pd.DataFrame(index=col_cat)
        stats_cat['Count'] = df[col_cat].count()
        stats_cat['Unique'] = df[col_cat].nunique()
        stats_cat['Top'] = df[col_cat].mode().iloc[0]
        stats_cat['Freq'] = df[col_cat].apply(lambda x: x.value_counts().iloc[0])
        resumo['Categóricas'] = stats_cat

    if len(col_date) > 0:
        stats_date = pd.DataFrame(index=col_date)
        stats_date['Count'] = df[col_date].count()
        stats_date['Min'] = df[col_date].min()
        stats_date['Max'] = df[col_date].max()
        resumo['Datas'] = stats_date

    return resumo


def exibir_resumo(resumo):
    print("\n" + "="*50)
    print("RESUMO DAS ESTATÍSTICAS DESCRITIVAS")
    print("="*50)

    for tipo, stats in resumo.items():
        print(f"\n{'='*20} {tipo.upper()} {'='*20}")
        try:
            from IPython.display import display
            display(stats.style.set_properties(**{
                'text-align': 'center'
            }).set_table_styles([{
                'selector': 'th',
                'props': [('text-align', 'center')]
            }]))
        except ImportError:
            print(stats)
        print("-"*50)


# ============================================================
# 📥 Carregamento seguro dos DataFrames
# ============================================================
def carregar_dataframe(caminho):
    obj = joblib.load(caminho)

    if isinstance(obj, pd.DataFrame):
        return obj

    if isinstance(obj, dict):
        # tenta encontrar uma chave com 'df' no nome
        for key in obj.keys():
            if 'df' in key.lower():
                return obj[key]
        # se não encontrar, tenta pegar o primeiro DataFrame
        for key, value in obj.items():
            if isinstance(value, pd.DataFrame):
                return value
        raise ValueError(f"Nenhum DataFrame encontrado no arquivo {caminho}. Chaves: {list(obj.keys())}")

    raise ValueError(f"Formato desconhecido no arquivo {caminho}. Tipo: {type(obj)}")


# ============================================================
# 🔗 Arquivos mapeados
# ============================================================
arquivos = {
    'df_unidades': 'df_unidades.z',
    'df_vendas': 'df_vendas.z',
    'df_categorias': 'df_categorias.z',
    'df_produtos': 'df_produtos.z'
}

# ============================================================
# 🚚 Carregar todos
# ============================================================
dataframes = {}

for nome, caminho in arquivos.items():
    try:
        df = carregar_dataframe(caminho)
        dataframes[nome] = df
        print(f"✅ {nome} carregado com sucesso. Shape: {df.shape}")
    except Exception as e:
        print(f"❌ Erro ao carregar {nome}: {e}")


# ============================================================
# 🚀 Geração de estatísticas
# ============================================================
for nome, df in dataframes.items():
    print(f"\n{'#'*60}")
    print(f"ESTATÍSTICAS PARA: {nome.upper()}")
    print(f"{'#'*60}")
    resumo = estatisticas_descritivas(df)
    exibir_resumo(resumo)


✅ df_unidades carregado com sucesso. Shape: (3, 3)
✅ df_vendas carregado com sucesso. Shape: (10418, 14)
✅ df_categorias carregado com sucesso. Shape: (50, 2)
✅ df_produtos carregado com sucesso. Shape: (9112, 9)

############################################################
ESTATÍSTICAS PARA: DF_UNIDADES
############################################################

RESUMO DAS ESTATÍSTICAS DESCRITIVAS

==================== NUMÉRICAS ====================


,Count,Mean,Std,Var,Min,25%,50% (Median),75%,Max
id,3,10002.000000,1.000000,1.000000,10001,10001.500000,10002.000000,10002.500000,10003


--------------------------------------------------

==================== CATEGÓRICAS ====================


,Count,Unique,Top,Freq
nome_fantasia,3,3,L01 - FORTALEZA,1
status,3,1,sim,3


--------------------------------------------------

############################################################
ESTATÍSTICAS PARA: DF_VENDAS
############################################################

RESUMO DAS ESTATÍSTICAS DESCRITIVAS

==================== NUMÉRICAS ====================


...

--------------------------------------------------

############################################################
ESTATÍSTICAS PARA: DF_PRODUTOS
############################################################

RESUMO DAS ESTATÍSTICAS DESCRITIVAS

==================== NUMÉRICAS ====================


,Count,Mean,Std,Var,Min,25%,50% (Median),75%,Max
codigo_categoria,9109,78.439000,128.770000,16581.588000,2.000000,3.000000,7.000000,64.000000,342.000000
preco_venda,9112,219.250000,442.911000,196170.088000,0.150000,22.307000,70.000000,218.345000,4485.820000
preco_padronizado,9112,-0.070000,0.318000,0.101000,-0.227000,-0.211000,-0.177000,-0.071000,2.990000
preco_normalizado,9112,0.004000,0.009000,0.000000,0.000000,0.000000,0.001000,0.004000,0.087000


--------------------------------------------------

==================== CATEGÓRICAS ====================


,Count,Unique,Top,Freq
id,9111,9111,000004,1
descricao,9112,9112,acessorio ref 10029,1
unidade,9112,5,un,9102
marca,9108,17,ediouro,1983
categoria,9109,19,livro,1983


--------------------------------------------------
